In [12]:
import numpy as np
from tqdm import tqdm, tqdm_notebook
from os import makedirs
from os.path import exists, join
from shutil import copyfile
from skimage import io

In [2]:
train_dir = "../LPD/training/"
test_dir = "../LPD/testing/"
validation_dir = "../LPD/validation/"

In [3]:
if not exists(train_dir):
    makedirs(train_dir)
if not exists(test_dir):
    makedirs(test_dir)
if not exists(validation_dir):
    makedirs(validation_dir)

In [4]:
x_train_paths = np.load("../Cache/LicensePlateDetection/x_train.npy")
x_test_paths = np.load("../Cache/LicensePlateDetection/x_test.npy")
x_validation_paths = np.load("../Cache/LicensePlateDetection/x_validation.npy")
y_train_paths = np.load("../Cache/LicensePlateDetection/y_train.npy")
y_test_paths = np.load("../Cache/LicensePlateDetection/y_test.npy")
y_validation_paths = np.load("../Cache/LicensePlateDetection/y_validation.npy")

In [24]:
print(x_train_paths[0], y_train_paths[0])
print(x_train_paths[0].split("/")[-1][:-4])
print(join(train_dir, x_train_paths[0].split("/")[-1][:-4]) + ".jpg")
im = io.imread(x_train_paths[0])
print(im.shape)
print(type(y_train_paths[0][0]))

../UFPR-ALPR-dataset/training/track0008/track0008[18].png [422 862  25  77]
track0008[18]
../LPD/training/track0008[18].jpg
(1080, 1920, 3)
<class 'numpy.int64'>


In [20]:
with open("test.txt", "w") as file:
    file.write("Text")

# Training

In [37]:
x = np.asarray(x_train_paths)
y = np.asarray(y_train_paths)
xy = np.column_stack((x, y))
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    bb = np.asarray(path[1:], dtype=int)
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    x = (bb[1] + bb[1] + bb[3]) / 2.0
    y = (bb[0] + bb[0] + bb[2]) / 2.0
    x = x * dw
    y = y * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    with open(join(train_dir, path[0].split("/")[-1][:-4]) + ".txt", "w") as file:
        file.write("0" + " " + " ".join([str(a) for a in bb]) + '\n')
    io.imsave(join(train_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)

HBox(children=(IntProgress(value=0, max=1800), HTML(value='')))

# Testing

In [38]:
x = np.asarray(x_test_paths)
y = np.asarray(y_test_paths)
xy = np.column_stack((x, y))
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    bb = np.asarray(path[1:], dtype=int)
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    x = (bb[1] + bb[1] + bb[3]) / 2.0
    y = (bb[0] + bb[0] + bb[2]) / 2.0
    x = x * dw
    y = y * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    with open(join(test_dir, path[0].split("/")[-1][:-4]) + ".txt", "w") as file:
        file.write("0" + " " + " ".join([str(a) for a in bb]) + '\n')
    io.imsave(join(test_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)

HBox(children=(IntProgress(value=0, max=1800), HTML(value='')))

# Validation

In [39]:
x = np.asarray(x_validation_paths)
y = np.asarray(y_validation_paths)
xy = np.column_stack((x, y))
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    bb = np.asarray(path[1:], dtype=int)
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    x = (bb[1] + bb[1] + bb[3]) / 2.0
    y = (bb[0] + bb[0] + bb[2]) / 2.0
    x = x * dw
    y = y * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    with open(join(validation_dir, path[0].split("/")[-1][:-4]) + ".txt", "w") as file:
        file.write("0" + " " + " ".join([str(a) for a in bb]) + '\n')
    io.imsave(join(validation_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)

HBox(children=(IntProgress(value=0, max=900), HTML(value='')))